In [ ]:
import os
import cv2
import json
import random
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt

from pycocotools.coco import COCO

# detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators

# Ignore warnings
import warnings
warnings.filterwarnings('ignore') 

# Setup logger
setup_logger()

In [ ]:
Data_Resister_training="train_xworld";
Data_Resister_valid="val_xworld";
from detectron2.data.datasets import register_coco_instances

register_coco_instances(Data_Resister_training,{}, 'dataset/train_xworld.json', Path("dataset/train_xworld"))
register_coco_instances(Data_Resister_valid,{},'dataset/val_xworld.json', Path("dataset/val_xworld"))

metadata = MetadataCatalog.get(Data_Resister_training)
dataset_train = DatasetCatalog.get(Data_Resister_training)
dataset_valid = DatasetCatalog.get(Data_Resister_valid)

In [ ]:
metadata

In [ ]:
for i in random.sample(range(len(dataset_train)), k=3):
    sample = dataset_train[i]

    fig, ax = plt.subplots(figsize =(12,8))
    img = cv2.imread(sample["file_name"])
    v = Visualizer(
            img[:, :, ::-1],
            metadata=metadata, 
            instance_mode=ColorMode.IMAGE_BW
        )
    
    for s in sample["annotations"]:
        if 'keypoints' in s.keys():
            s.pop("keypoints", None)
            
    out = v.draw_dataset_dict(sample)
    ax.axis('off')
    print(f"image name:{sample['file_name']}")
    ax.imshow(out.get_image()[:, :, ::-1])

In [ ]:
class Trainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return DatasetEvaluators([COCOEvaluator(dataset_name, output_dir=output_folder)])

In [ ]:
cfg = get_cfg()
config_name = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml" 
cfg.merge_from_file(model_zoo.get_config_file(config_name))

cfg.DATASETS.TRAIN = (Data_Resister_training,)
cfg.DATASETS.TEST = (Data_Resister_valid,)

cfg.MODEL.WEIGHTS ="detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x/137260431/model_final_a54504.pkl"
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8 
cfg.MODEL.MASK_ON = True
cfg.MODEL.DEVICE = "cuda:0"


cfg.DATALOADER.NUM_WORKERS = 8

cfg.SOLVER.IMS_PER_BATCH = 8 
cfg.SOLVER.BASE_LR = 0.01 
cfg.SOLVER.WARMUP_ITERS = 10 
cfg.SOLVER.MAX_ITER = 5000

cfg.SOLVER.STEPS = (500, 1000) 
cfg.SOLVER.CHECKPOINT_PERIOD=1000

cfg.TEST.EVAL_PERIOD = 1000

cfg.OUTPUT_DIR = "./segmentation"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()